<a href="https://colab.research.google.com/github/Swayamprakashpatel/DD/blob/main/FIngerprint_SMILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title DOWNLOAD DATA FROM KAGGLE
# DOWNLOAD DATA FROM KAGGLE (!IMPORTANT!: REFRESH RUNTIME BEFORE RE-RUNNING THE CODE)
#%%capture
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

#Make Directory of Kaggle and set its permission for access.
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

# Download Data from Kaggle Fast and Unzip them in /content
!kaggle datasets download -d drswayamprakashpatel/DD-Dataset-csv  -p /content # For model download

#Unzip data (Two Folders - Training and Validation)
import os
os.chdir('/content')
#create a directory named train/
!unzip -q DD-Dataset-csv.zip #Unzip Model

Saving kaggle.json to kaggle.json
kaggle.json
 89% 5.00M/5.59M [00:00<00:00, 49.8MB/s]
100% 5.59M/5.59M [00:00<00:00, 54.8MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Load the dataset
data = pd.read_csv('/content/DATASET_Final.csv')  # Replace with the path to your dataset
#data = data.iloc[0:10000,:]
# Split the data into input and output columns
X = data.iloc[:, 7:888]  # Input columns
y = data['SMILE']      # Output column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reshape the input data
X_train_reshaped = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_reshaped = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Preprocess the output data (SMILES)
tokenizer = Tokenizer(char_level= True) #char_level True tokenize every character like '=', '@', etc. 
tokenizer.fit_on_texts(y_train)
y_train_sequences = tokenizer.texts_to_sequences(y_train)
y_test_sequences = tokenizer.texts_to_sequences(y_test)

# Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in y_train_sequences + y_test_sequences)

# Pad sequences to ensure consistent length
y_train_padded = pad_sequences(y_train_sequences, maxlen=max_sequence_length, padding = 'post')
y_test_padded = pad_sequences(y_test_sequences, maxlen=max_sequence_length, padding = 'post')

# Define the model
input_shape = (1, X_train.shape[1])
output_vocab_size = len(tokenizer.word_index) + 1
hidden_units = 512

inputs = Input(shape=input_shape)

##ENCODER###
encoder = LSTM(hidden_units, return_sequences= False)(inputs) #return_sequences=True in the first LSTM layer (encoder). This will ensure that the output of the first LSTM layer is a sequence, which can then be passed as input to the second LSTM layer.
#encoder_1 = LSTM(hidden_units, return_sequences=False)(encoder) # Extra LSTM Layer: 
hidden_layer = Dense(hidden_units, activation='tanh')(encoder)# Hidden layer in encoder

##DECODER##
decoder = Dense(hidden_units, activation='tanh')(hidden_layer)
hidden_layer_1 = Dense(hidden_units, activation='tanh')(decoder) #Hidden laye rin decoder
outputs = Dense(max_sequence_length, activation='tanh')(hidden_layer_1)  # Updated output layer units


model = Model(inputs=inputs, outputs=outputs)

#Train Model

import tensorflow as tf
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

filepath = '/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5'

monitor = 'val_accuracy'
mode = 'max'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor = monitor, mode = mode, save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor= monitor, mode = mode, patience=250, verbose =1), [tensorboard_callback]]


model.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
hist = model.fit(X_train_reshaped, y_train_padded, epochs= 2000, callbacks=[checkpoint],validation_data=(X_test_reshaped, y_test_padded), batch_size= None)




Epoch 1/2000
684/684 [==============================] - ETA: 0s - loss: 1880.2861 - accuracy: 0.0029
Epoch 1: val_accuracy improved from -inf to 0.00416, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 35s 48ms/step - loss: 1880.2861 - accuracy: 0.0029 - val_loss: 1883.4935 - val_accuracy: 0.0042
Epoch 2/2000
684/684 [==============================] - ETA: 0s - loss: 1867.6450 - accuracy: 0.0032
Epoch 2: val_accuracy improved from 0.00416 to 0.00438, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 34s 50ms/step - loss: 1867.6450 - accuracy: 0.0032 - val_loss: 1869.0331 - val_accuracy: 0.0044
Epoch 3/2000
684/684 [==============================] - ETA: 0s - loss: 1849.4357 - accuracy: 0.0041
Epoch 3: val_accuracy improved from 0.00438 to 0.00470, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 31s 46ms/step

KeyboardInterrupt: ignored

In [ ]:
print(y_train_padded.shape)
print(X_train.shape)
print(y_train_padded[0:1,:])

X = data.iloc[0:5, 7:888]  # Input columns
X_reshaped = X.values.reshape(X.shape[0], 1, X.shape[1])

predict = np.round(model.predict(X_reshaped))
predict_round = np.round(model.predict(X_reshaped))
original_sequences = tokenizer.sequences_to_texts(predict_round)

# Print the original sequences
print(predict)
print(predict_round)
print(original_sequences)

(21863, 1000)
(21863, 881)
[[ 1  1  3  1  4  1  3  1  3  2  5  4  5  4  6 12  3  2  5  4  3  2  5  4
   1  7  2  1  1  2  1  3  1  2  1  7  4  1  8  2  1  1  2  1  3  1  2  1
   8  4 24 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 

In [ ]:
print(original_sequences)

In [ ]:


#FOR PREDICTION
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5')

# Prepare the input data
input_data = X_test.iloc[0].values  # Replace with your input data of length 881
input_reshaped = input_data.reshape(1, 1, len(input_data))

# Make predictions
predictions = model.predict(input_reshaped)

# Decode the predicted sequence
predicted_sequence = tokenizer.sequences_to_texts(predictions.argmax(axis=2))[0]

**DIFFERNT APPORACH: SIMPLE MACHINE LEARNING**
881 INPUT and SMILES are tokenized and padded as output.



In [ ]:
# different approach

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint

# Load the dataset
data = pd.read_csv('/content/DATASET_Final.csv')  # Replace with the path to your dataset
#data = data.iloc[0:10,:]
# Split the data into input and output columns
X = data.iloc[:, 7:888]  # Input columns
y = data['SMILE']  # Output column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Preprocess the output data (SMILES)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(y_train)
y_train_sequences = tokenizer.texts_to_sequences(y_train)
y_test_sequences = tokenizer.texts_to_sequences(y_test)

# Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in y_train_sequences + y_test_sequences)

# Pad sequences to ensure consistent length
y_train_padded = pad_sequences(y_train_sequences, maxlen=max_sequence_length, padding = 'post') #can put fixed values like 100, 500 or 1000 in maxlen
y_test_padded = pad_sequences(y_test_sequences, maxlen=max_sequence_length, padding = 'post')



# Define the model
input_shape = (X_train.shape[1],)
output_classes = y_train_padded.shape[1]

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(881, activation='tanh', input_shape= input_shape),
                             tf.keras.layers.Dense(881, activation='tanh'),
                             #tf.keras.layers.Dense(512, activation='tanh'),
                             #tf.keras.layers.Dense(512, activation='tanh'),
                             tf.keras.layers.Dense(output_classes, activation= 'exponential')])

# Compile and Train the model
import tensorflow as tf
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

filepath = '/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5'

#model = tf.keras.models.load_model('/content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5') # run this line when model training needs to be continued from where it was stopped. 

monitor = 'val_accuracy'
mode = 'max'
 
checkpoint = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor = monitor, mode = mode, save_best_only=True, Save_weights_only = False, verbose = 1), 
              tf.keras.callbacks.EarlyStopping(monitor= monitor, mode = mode, patience=250, verbose =1), [tensorboard_callback]]


model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.1), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics=['accuracy'])


hist = model.fit(X_train, y_train_padded, epochs= 2000, callbacks=[checkpoint],validation_data=(X_test, y_test_padded), batch_size= None)


Epoch 1/2000
671/684 [============================>.] - ETA: 0s - loss: 929.2239 - accuracy: 0.0273
Epoch 1: val_accuracy improved from -inf to 0.02401, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 6s 7ms/step - loss: 928.7890 - accuracy: 0.0274 - val_loss: 913.7451 - val_accuracy: 0.0240
Epoch 2/2000
676/684 [============================>.] - ETA: 0s - loss: 899.2921 - accuracy: 0.0380
Epoch 2: val_accuracy improved from 0.02401 to 0.03671, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 5s 7ms/step - loss: 897.8591 - accuracy: 0.0380 - val_loss: 902.1858 - val_accuracy: 0.0367
Epoch 3/2000
680/684 [============================>.] - ETA: 0s - loss: 891.1694 - accuracy: 0.0435
Epoch 3: val_accuracy improved from 0.03671 to 0.04013, saving model to /content/drive/MyDrive/Model_DE/PF_SML_Model.hdf5
684/684 [==============================] - 4s 6ms/step - loss: 891.

In [ ]:
original_sequences = tokenizer.sequences_to_texts(y_train_padded[0:1,:])

# Print the original sequences
print(original_sequences)

['c n 1 c ( = c ( n = c 1 n ) c c 2 = c c 3 = c ( c = c 2 ) o c o 3 ) c c 4 = c c 5 = c ( c = c 4 ) o c o 5']


In [ ]:
# Load the dataset
data = pd.read_csv('/content/DATASET_Final.csv')  # Replace with the path to your dataset
data = data.iloc[0:1,:]
# Split the data into input and output columns
X = data.iloc[:, 7:888]  # Input columns
y = data['SMILE']  # Output column


# Preprocess the output data (SMILES)
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(y_train)
y_token = tokenizer.texts_to_sequences(y)


In [ ]:
print(y)
print(y_token)

0    CN1CCN(CC1)CC(=O)N(C)C2=CC=C(C=C2)N=C(C3=CC=CC...
Name: SMILE, dtype: object
[[1, 6, 7, 1, 1, 6, 3, 1, 1, 7, 4, 1, 1, 3, 2, 5, 4, 6, 3, 1, 4, 1, 8, 2, 1, 1, 2, 1, 3, 1, 2, 1, 8, 4, 6, 2, 1, 3, 1, 9, 2, 1, 1, 2, 1, 1, 2, 1, 9, 4, 1, 10, 2, 1, 3, 6, 1, 11, 2, 1, 10, 1, 2, 1, 1, 3, 2, 1, 11, 4, 1, 3, 2, 5, 4, 5, 1, 4, 5]]


In [ ]:
X = data.iloc[0:1, 7:888]  # Input columns

predict = np.round(model.predict(X))
print (predict)


original_sequences = tokenizer.sequences_to_texts(predict)

# Print the original sequences
print(original_sequences)